In [1]:
import numpy as np 
import pandas as pd 
import warnings
warnings.filterwarnings("ignore")

#import src
import Cleanup
import Transformation
import FileProcessing

In [2]:
#define the objects to be called later. 
cleanup=Cleanup.Cleanup()
transform=Transformation.Transformation()
fileProcessing=FileProcessing.FileProcessing()

Read the File from Data folder and remove all duplicates. 

In [3]:
file_name='Next_Generation_Simulation__NGSIM__Vehicle_Trajectories_and_Supporting_Data'
ngsim=fileProcessing.read_input(file_name)

original File path: c:\Users\StudentAccount\Python\Capstone\gitcodelocation\DataDrivenCarFollowing\datadrivencarfollowing-v1\scripts
Data File path: c:\Users\StudentAccount\Python\Capstone\gitcodelocation\DataDrivenCarFollowing\datadrivencarfollowing-v1\data


Remove any Duplicate records from the original File. 

In [4]:
cleanup.remove_dups(ngsim)

704000 duplicate values have been removed


,Vehicle_ID,Frame_ID,Total_Frames,Global_Time,Local_X,Local_Y,Global_X,Global_Y,v_length,v_Width,...,D_Zone,Int_ID,Section_ID,Direction,Movement,Preceding,Following,Space_Headway,Time_Headway,Location
0,515,2330,1123,1118848075000,30.034,188.062,6451203.729,1873252.549,13.0,6.9,...,NaN,NaN,NaN,NaN,NaN,500,523,119.10,5.11,us-101
2,2224,6548,1902,1113437421700,41.429,472.901,6042814.264,2133542.012,14.3,6.9,...,NaN,NaN,NaN,NaN,NaN,2208,2211,53.34,2.01,i-80
3,2127,6459,567,1118847624800,19.632,1775.614,6452425.122,1872172.475,13.5,6.9,...,NaN,NaN,NaN,NaN,NaN,2124,2132,48.92,1.30,us-101
4,1033,4827,592,1118848324700,6.202,1701.144,6452347.673,1872258.452,13.5,4.4,...,NaN,NaN,NaN,NaN,NaN,1029,1040,38.81,0.92,us-101
6,1890,9157,628,1118849672700,53.514,817.521,6451655.238,1872800.663,24.0,8.5,...,NaN,NaN,NaN,NaN,NaN,1882,1897,102.65,2.27,us-101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11850521,1355,5622,938,1113438127100,18.275,1464.336,6042648.149,2134520.896,12.3,6.8,...,NaN,NaN,NaN,NaN,NaN,1351,1361,26.91,15.92,i-80
11850522,1474,5688,401,1113438133700,5.509,560.412,6042767.840,2133624.549,17.8,7.9,...,NaN,NaN,NaN,NaN,NaN,1468,1480,74.49,1.73,i-80
11850523,398,2368,654,1113437801700,18.903,635.841,6042771.741,2133701.076,15.2,8.5,...,NaN,NaN,NaN,NaN,NaN,385,406,90.95,3.77,i-80
11850524,599,875,577,1113436854400,77.094,621.358,6042831.282,2133693.854,15.3,6.4,...,NaN,NaN,NaN,NaN,NaN,0,611,0.00,0.00,i-80


Feet to Metres:

Below Four parametrs consider distance in Feet and they are converted to metre. Speed to Metre/ second from Feet/second and acceleration from feet/second Square to Metre/Second Square. 

Drop the columns we wont use in the model.
Add Relative time which is the time from the first timframe in the dataset. This is calculated by using the minimum value ofGlobal time(absolute time from 1970)

In [5]:
ngsim=transform.convert_feet_to_metre(ngsim)

In [6]:
ngsim.shape

(11146526, 25)

In [7]:
ngsim=transform.drop_not_required_columns(ngsim)

Create Placeholders for the user defined columns we will be populating later, to be used in the model. 
Map the Vehicle classes to their english equivalent. 
Filter the original file into US-101 and I-80 to perform and populate the required fields later. These files contain same values of Vehicle ID used for multiple vehicles based on the highway. 



In [8]:
#filtered_ngsim= ngsim[((ngsim['Location'] == 'us-101') | (ngsim['Location'] == 'i-80')) & (ngsim['Preceding']>0) & (ngsim['Following']>0) ]
ngsim=transform.create_column_placeholders(ngsim)

Split data into US-101 and I-80 specific dataframes.

In [9]:
filtered_ngsim_U,filtered_ngsim_I = transform.bifurcate_highways(ngsim)

Create Sets to use and create the L-F Pairs. 

In [10]:
v_Class_M_U,v_Class_C_U,v_Class_HV_U = transform.class_vehicle_sets(filtered_ngsim_U)
v_Class_M_I,v_Class_C_I,v_Class_HV_I = transform.class_vehicle_sets(filtered_ngsim_I)


Add the Preceding Vehicle Class to the new previous class column

In [11]:
filtered_ngsim_U=transform.preceding_vehicle_class(filtered_ngsim_U,v_Class_M_U,v_Class_C_U,v_Class_HV_U)
filtered_ngsim_I=transform.preceding_vehicle_class(filtered_ngsim_I,v_Class_M_I,v_Class_C_I,v_Class_HV_I)

Map the Vehicle Length of the previous vehicle into the row. 

In [12]:
filtered_ngsim_U=transform.preceding_vehicle_length(filtered_ngsim_U)
filtered_ngsim_I=transform.preceding_vehicle_length(filtered_ngsim_I)

Convert the Front to Front Space Headway to Back Bumper (preceding Vehicle) to front Bumper(of Following ) Headway. This will give the actual distance between the vehicle ignoring the vehicle length.
Find the time it would take for the Following vehicle to reach the rear Bumper of the Preceding Vehicle. 

In [13]:
filtered_ngsim_U= transform.front_to_front_bumper_details_changed_to_rear_to_front_bumper_details(filtered_ngsim_U)
filtered_ngsim_I= transform.front_to_front_bumper_details_changed_to_rear_to_front_bumper_details(filtered_ngsim_I)

In [14]:
filtered_ngsim_U.columns

Index(['Vehicle_ID', 'Frame_ID', 'Global_Time', 'Local_X', 'Local_Y',
       'v_length', 'v_Class', 'v_Vel', 'v_Acc', 'Lane_ID', 'Preceding',
       'Following', 'Space_Headway', 'Time_Headway', 'Location',
       'Preceding_Vehicle_Class', 'Rear_to_Front_Space_Headway',
       'Front_To_Rear_Time_Headway', 'Velocity Difference_Following-Preceding',
       'Acceleration Difference_Following-Preceding', 'L-F_Pair',
       'v_Class_Name', 'Vehicle_combination', 'preceding_vehicle_length'],
      dtype='object')

Vehicle Combination of the Preceding and the Following Vehicle. 

In [15]:
filtered_ngsim_U=transform.map_preceding_vehicle_details(filtered_ngsim_U)
filtered_ngsim_I=transform.map_preceding_vehicle_details(filtered_ngsim_I)

us-101::621 cars dont change lanes
us-101::2226 cars Change lanes
i-80::817 cars dont change lanes
i-80::2184 cars Change lanes


In [16]:
filtered_ngsim_U=transform.map_pairs(filtered_ngsim_U)
filtered_ngsim_I=transform.map_pairs(filtered_ngsim_I)

filtered_ngsim_U=cleanup.filter_records_for_model(filtered_ngsim_U)
filtered_ngsim_I=cleanup.filter_records_for_model(filtered_ngsim_I)

filtered_ngsim_U.drop(columns=["total_pair_duration",'pair_Time_Duration'],axis=1,inplace=True)
filtered_ngsim_I.drop(columns=["total_pair_duration",'pair_Time_Duration'],axis=1,inplace=True)
filtered_ngsim_U=transform.map_pairs(filtered_ngsim_U)
filtered_ngsim_I=transform.map_pairs(filtered_ngsim_I)

dataset before Row Removal(4098933, 33)
us-101: 157194 vehicles first and Last 5 seconds removed from Lane 1 due to lane changing
us-101: 10594 vehicles first and Last 5 seconds removed from Lane 2 due to lane changing
us-101: 8027 vehicles first and Last 5 seconds removed from Lane 3 due to lane changing
us-101: 530853 have multiple lengths and classes for same Vehicle ID
us-101: 780619 have total car following less than 30 seconds
us-101: 376267 have time Headway less than 5 seconds
us-101: 1598920 have Lane ID as 7 or 8 which are the Ramps
us-101: 2455598 rows removed using above criterias
dataset before Row Removal(4566387, 33)
i-80: 155906 vehicles first and Last 5 seconds removed from Lane 1 due to lane changing
i-80: 7461 vehicles first and Last 5 seconds removed from Lane 2 due to lane changing
i-80: 6096 vehicles first and Last 5 seconds removed from Lane 3 due to lane changing
i-80: 830630 have multiple lengths and classes for same Vehicle ID
i-80: 940574 have total car follo


1. Velocity difference of the Lead minus Preceding Vehicle.
2. Acceleration difference of the Lead minus Preceding Vehicle.
3. Find the Pair details of the Lead_following 
4. Find the cumulative time the Pair runs within the highway as a pair. 

In [17]:
filtered_ngsim= fileProcessing.merge_files(filtered_ngsim_U , filtered_ngsim_I)
filtered_ngsim=cleanup.remove_dup_pairs(filtered_ngsim)
fileProcessing.export_file(filtered_ngsim,'Cleaned_NGSIM_Data')

 Merged Record Count:2634976, df1:1643335, df2:991641
0 rows removed for Dups. Count Before removal: 2634976, Count After Removal:2634976


True

In [18]:
filtered_ngsim.columns

Index(['Vehicle_ID', 'Frame_ID', 'Global_Time', 'Local_X', 'Local_Y',
       'v_length', 'v_Class', 'v_Vel', 'v_Acc', 'Lane_ID', 'Preceding',
       'Following', 'Space_Headway', 'Time_Headway', 'Location',
       'Preceding_Vehicle_Class', 'Rear_to_Front_Space_Headway',
       'Front_To_Rear_Time_Headway', 'Velocity Difference_Following-Preceding',
       'Acceleration Difference_Following-Preceding', 'L-F_Pair',
       'v_Class_Name', 'Vehicle_combination', 'preceding_vehicle_length',
       'lane_changes', 'Prec_Vehicle_ID', 'preceding_Vehicle_Velocity',
       'preceding_Vehicle_Acceleration', 'preceding_car_lane_changes',
       'preceding_Local_Y', 'preceding_v_Class', 'pair_Time_Duration',
       'total_pair_duration'],
      dtype='object')

In [19]:
print(f"{filtered_ngsim[(filtered_ngsim['preceding_car_lane_changes'] == True) ]['Vehicle_ID'].unique().size} cars Change lanes")
print(f"{filtered_ngsim[(filtered_ngsim['preceding_car_lane_changes'] == False) ]['Vehicle_ID'].unique().size} cars DonT")

2171 cars Change lanes
833 cars DonT


In [20]:
#filtered_ngsim[((filtered_ngsim['Vehicle_ID'] <= 100 )  ) & ((filtered_ngsim['Preceding'] <= 100 )  )].to_csv('classVERIFYFILE.CSV', index = False)

In [21]:
filtered_ngsim

,Vehicle_ID,Frame_ID,Global_Time,Local_X,Local_Y,v_length,v_Class,v_Vel,v_Acc,Lane_ID,...,preceding_vehicle_length,lane_changes,Prec_Vehicle_ID,preceding_Vehicle_Velocity,preceding_Vehicle_Acceleration,preceding_car_lane_changes,preceding_Local_Y,preceding_v_Class,pair_Time_Duration,total_pair_duration
0,2,13,1118846980200,5.019142,10.784129,4.41960,2,12.192000,0.000000,2,...,0.00000,True,0,0.000000,0.000000,False,9999.000000,0.0,0.0,88.1
1,2,14,1118846980300,5.013046,12.003329,4.41960,2,12.192000,0.000000,2,...,0.00000,True,0,0.000000,0.000000,False,9999.000000,0.0,0.1,88.1
2,2,15,1118846980400,5.006645,13.222529,4.41960,2,12.192000,0.000000,2,...,0.00000,True,0,0.000000,0.000000,False,9999.000000,0.0,0.2,88.1
3,2,16,1118846980500,5.000244,14.441424,4.41960,2,12.192000,0.000000,2,...,0.00000,True,0,0.000000,0.000000,False,9999.000000,0.0,0.3,88.1
4,2,17,1118846980600,4.994148,15.660929,4.41960,2,12.192000,0.000000,2,...,0.00000,True,0,0.000000,0.000000,False,9999.000000,0.0,0.4,88.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
991636,2480,11235,1113438688400,9.011412,473.917670,4.35864,2,7.040880,0.000000,3,...,4.61264,True,2483,2.505456,0.606552,True,491.075777,2.0,32.2,32.6
991637,2480,11236,1113438688500,9.031834,474.621758,4.35864,2,7.040880,0.000000,3,...,4.61264,True,2483,2.545080,0.018288,True,491.333333,2.0,32.3,32.6
991638,2480,11237,1113438688600,9.052255,475.325846,4.35864,2,7.040880,0.000000,3,...,4.61264,True,2483,2.551176,0.000000,True,491.588755,2.0,32.4,32.6
991639,2480,11238,1113438688700,9.072372,476.029020,4.35864,2,7.046976,0.124968,3,...,4.61264,True,2483,2.554224,0.118872,True,491.842958,2.0,32.5,32.6


In [22]:
filtered_ngsim['L-F_Pair'].unique().shape

(4817,)

In [23]:
filtered_ngsim[((filtered_ngsim['Space_Headway'] == 0 ) & (filtered_ngsim['Preceding'] > 0 )) ]


,Vehicle_ID,Frame_ID,Global_Time,Local_X,Local_Y,v_length,v_Class,v_Vel,v_Acc,Lane_ID,...,preceding_vehicle_length,lane_changes,Prec_Vehicle_ID,preceding_Vehicle_Velocity,preceding_Vehicle_Acceleration,preceding_car_lane_changes,preceding_Local_Y,preceding_v_Class,pair_Time_Duration,total_pair_duration
456259,2964,8731,1118847852000,1.483766,94.405399,5.02920,2,2.618232,-3.413760,1,...,4.72440,False,2956,1.411224,1.298448,False,94.406923,2.0,4.7,57.5
570284,2491,7916,1113437558500,5.230673,182.539843,4.66344,2,3.453384,0.036576,2,...,4.51104,True,2486,3.200400,0.000000,False,182.540148,2.0,29.3,89.6
833036,1748,6068,1113438171700,5.372100,22.727107,5.42544,2,7.504176,0.000000,2,...,4.35864,True,1734,7.620000,0.000000,True,22.727412,2.0,0.2,29.7


In [24]:
filtered_ngsim.columns

Index(['Vehicle_ID', 'Frame_ID', 'Global_Time', 'Local_X', 'Local_Y',
       'v_length', 'v_Class', 'v_Vel', 'v_Acc', 'Lane_ID', 'Preceding',
       'Following', 'Space_Headway', 'Time_Headway', 'Location',
       'Preceding_Vehicle_Class', 'Rear_to_Front_Space_Headway',
       'Front_To_Rear_Time_Headway', 'Velocity Difference_Following-Preceding',
       'Acceleration Difference_Following-Preceding', 'L-F_Pair',
       'v_Class_Name', 'Vehicle_combination', 'preceding_vehicle_length',
       'lane_changes', 'Prec_Vehicle_ID', 'preceding_Vehicle_Velocity',
       'preceding_Vehicle_Acceleration', 'preceding_car_lane_changes',
       'preceding_Local_Y', 'preceding_v_Class', 'pair_Time_Duration',
       'total_pair_duration'],
      dtype='object')